In [2]:
%reload_ext autoreload
%autoreload 2

from api import *
from dataframe import *
from database import *
from tables import *

# Database connection

In [3]:
engine, session = connect()

2022-11-21 20:56:34,492 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-11-21 20:56:34,492 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-11-21 20:56:34,494 INFO sqlalchemy.engine.Engine select current_schema()
2022-11-21 20:56:34,494 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-11-21 20:56:34,495 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-11-21 20:56:34,496 INFO sqlalchemy.engine.Engine [raw sql] {}


# Load CSV files

In [4]:
original_filename = 'csv/original.csv'
george_filename = 'csv/george_data.csv'

df = pandas.read_csv(original_filename, sep=';', low_memory=False, skipinitialspace=True)
df_george = pandas.read_csv(george_filename, sep=';', low_memory=False, skipinitialspace=True)

## Create table County and download e inserted info of API

In [6]:
create_table_if_not_exists(engine, County)

count_records_in_county = session.query(County).count()

if count_records_in_county == 0:
    response = get_data_api()
    for i, county in enumerate(response.json()):
        session.add(create_county(county))
        session.commit()

2022-11-21 20:56:35,221 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-21 20:56:35,224 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT county.id AS county_id, county.county AS county_county, county.uf AS county_uf, county.uf_name AS county_uf_name 
FROM county) AS anon_1
2022-11-21 20:56:35,225 INFO sqlalchemy.engine.Engine [generated in 0.00079s] {}


## Create table DataSP and insert rows of sheet in db

In [7]:
create_table_if_not_exists(engine, DataTrustedIdentifier)

count_records_in_data_sp = session.query(DataSP).count()

if count_records_in_data_sp == 0:
    rename_header_dataframe(df)
    df = preprocess(df)
    for row in df.iterrows():
        try:
            data_sp = create_datasp(row[1])
            session.add(data_sp)
        except:
            session.commit()
            session.flush()

2022-11-21 20:56:35,251 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT data.seq AS data_seq, data.modified AS data_modified, data.institution_code AS data_institution_code, data.collection_code AS data_collection_code, data.catalog_number AS data_catalog_number, data.basis_of_record AS data_basis_of_record, data.kingdom AS data_kingdom, data.phylum AS data_phylum, data.classe AS data_classe, data."order" AS data_order, data.family AS data_family, data.genus AS data_genus, data.specific_epithet AS data_specific_epithet, data.infraspecific_epithet AS data_infraspecific_epithet, data.scientific_name AS data_scientific_name, data.scientific_name_authorship AS data_scientific_name_authorship, data.identified_by AS data_identified_by, data.year_identified AS data_year_identified, data.month_identified AS data_month_identified, data.day_identified AS data_day_identified, data.type_status AS data_type_status, data.recorded_by AS data_recorded_by, data.record_number AS d

## Adds 'yes' in the George column ('yes' are records marked as reliable by George))

In [8]:
count_records_marked_by_george = session.query(DataSP)\
    .filter(DataSP.george == True)\
    .count()

if count_records_marked_by_george == 0:
    for row in df_george.iterrows():
        get_value_column_george = row[1]['GEORGE']
        if get_value_column_george.lower() == 'sim':
            session.query(DataSP)\
                .filter(DataSP.seq == row[1]['seq'])\
                .update({'george': True}, synchronize_session=False)

2022-11-21 20:56:35,273 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT data.seq AS data_seq, data.modified AS data_modified, data.institution_code AS data_institution_code, data.collection_code AS data_collection_code, data.catalog_number AS data_catalog_number, data.basis_of_record AS data_basis_of_record, data.kingdom AS data_kingdom, data.phylum AS data_phylum, data.classe AS data_classe, data."order" AS data_order, data.family AS data_family, data.genus AS data_genus, data.specific_epithet AS data_specific_epithet, data.infraspecific_epithet AS data_infraspecific_epithet, data.scientific_name AS data_scientific_name, data.scientific_name_authorship AS data_scientific_name_authorship, data.identified_by AS data_identified_by, data.year_identified AS data_year_identified, data.month_identified AS data_month_identified, data.day_identified AS data_day_identified, data.type_status AS data_type_status, data.recorded_by AS data_recorded_by, data.record_number AS d

## Close connection

In [ ]:
session.close()
engine.dispose()